#### onnx生成

搭建onnx

In [ ]:
import onnx
from onnx import helper
from onnx import TensorProto
import numpy as np

### 搭建onnx模型
weight = np.random.randn(36)
X = helper.make_tensor_value_info('X', TensorProto.FLOAT, [1, 2, 4, 4])
W = helper.make_tensor('W', TensorProto.FLOAT, [2, 2, 3, 3], weight)
B = helper.make_tensor('B', TensorProto.FLOAT, [2], [1.0, 2.0])
Y = helper.make_tensor_value_info('Y', TensorProto.FLOAT, [1, 2, 2, 2])
node_def = helper.make_node(
'Conv', # node name
['X', 'W', 'B'],
['Y'], # outputs
# attributes
strides=[2,2],
)
graph_def = helper.make_graph(
[node_def],
'test_conv_mode',
[X], # graph inputs
[Y], # graph outputs
initializer=[W, B],
)
mode_def = helper.make_model(graph_def, producer_name='onnx-example')
onnx.checker.check_model(mode_def)
onnx.save(mode_def, "./Conv.onnx")

通过现有框架生成onnx

In [ ]:
import torch,onnx
import sys,os
sys.path.append(r'D:\31890\Desktop\codefile\Utils')
from model__ import U_Net_o


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = U_Net_o(2)
net = net.eval().to(device)
net.load_state_dict(torch.load(r'D:\31890\Desktop\codefile\Utils\Packages\onnx__\unet.pth',map_location=device)) 

## 生成onnx模型

x = torch.randn(1, 3, 512, 512)
torch_out = torch.onnx.export(net,
                            x,
                            "test.onnx",
                            input_names=['input'],
                            output_names=['output'],
                            opset_version=11  #算子版本
                            )

In [ ]:
import onnx
from onnx.external_data_helper import load_external_data_for_model

## 加载 onnx 模型
onnx_model = onnx.load(r'D:\31890\Desktop\codefile\Utils\Packages\onnx__\test.onnx') 
## 加载模型和外部数据
onnx_model = onnx.load('path/to/the/model.onnx', load_external_data=False)
load_external_data_for_model(onnx_model, 'data/directory/path/')
# Save the ONNX model
onnx.save(onnx_model, 'path/to/the/model.onnx')

验证模型是否导出成功

In [ ]:
import onnx
net  = onnx.load(r"D:\31890\Desktop\codefile\Utils\Packages\onnx__\test.onnx")
## 验证格式是否正确
onnx.checker.check_model(net)
## 打印计算图
# print(onnx.helper.printable_graph(net.graph) )

#### onnx推理单张

In [ ]:
import onnxruntime
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image

In [ ]:
## 加载模型
t_session = onnxruntime.InferenceSession(r'D:\31890\Desktop\codefile\Utils\Packages\onnx__\test.onnx')

In [ ]:
img_path = r'D:\31890\Desktop\codefile\Utils\Packages\onnx__\test.jpg'
img_pil = Image.open(img_path)
transform = transforms.Compose([
    transforms.Resize(512),
    transforms.ToTensor(),
    transforms.Normalize(
    mean=[0.4754358, 0.35509014, 0.282971],
    std=[0.16318515, 0.15616792, 0.15164918])
    ])
input_tensor = transform(img_pil).unsqueeze(0).numpy()
# print(input_tensor.shape)

In [ ]:
# ONNX Runtime 输入
ort_inputs = {'input': input_tensor}
# ONNX Runtime 输出
pred_logits = t_session.run(['output'], ort_inputs)[0]
pred_logits = torch.tensor(pred_logits)

In [ ]:
pred_logits = F.softmax(pred_logits[0],dim=0)
pred = np.array(pred_logits.argmax(axis=0),)*255

In [ ]:
pred = Image.fromarray(pred.astype(np.uint8))
pred.save(r'D:\31890\Desktop\codefile\Utils\Packages\onnx__\test2.png')

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(pred,cmap='gray')     # 加载图像
plt.axis('on')                  # 是否加载坐标轴
plt.title('image')              # 图像标题
plt.show()     

#### onnx 推理大图

In [1]:
import onnxruntime
import numpy as np
import torch,cv2,time,tqdm
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

In [3]:
# 加载
t_session = onnxruntime.InferenceSession(r'D:\31890\Desktop\codefile\Utils\Packages\onnx__\test.onnx')

# 多核加速
# cpu_num_thread = 4
# t_session.intra_op_num_threads = cpu_num_thread
# t_session.execution_mode = onnxruntime.ExecutionMode.ORT_SEQUENTIAL

time_start = time.time()
img_path = r'D:\31890\Desktop\codefile\Utils\Packages\onnx__\test.jpg'
img_pil = Image.open(img_path)

# 输出图像的尺寸
pixel_shape = 512
origin_w, origin_h = img_pil.size
# 新的高和宽
new_h = int((origin_h//pixel_shape+1)*pixel_shape)
new_w = int((origin_w//pixel_shape+1)*pixel_shape)
# 多余的尺寸
h_padding = new_h-origin_h
w_padding = new_w-origin_w

# 创建
img_new = Image.new('RGB',(new_w,new_h))
img_new.paste(img_pil, (w_padding, h_padding))  ##上左田间黑边
mask = np.zeros((new_h,new_w))

transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(
            mean=[0.4754358, 0.35509014, 0.282971],
            std=[0.16318515, 0.15616792, 0.15164918])
            ])

img_new = transform(img_new).unsqueeze(0).numpy()

for u in tqdm.tqdm(range(origin_h//pixel_shape)):
    for v in range(origin_w//pixel_shape):
        x = pixel_shape * u
        y = pixel_shape * v

        input_tensor = img_new[:,:,x : x + pixel_shape, y : y + pixel_shape]
        # ONNX Runtime 输入
        ort_inputs = {'input': input_tensor}
        # ONNX Runtime 输出
        pred_logits = t_session.run(['output'], ort_inputs)[0]
        pred_logits = torch.tensor(pred_logits)
        pred_logits = F.softmax(pred_logits[0],dim=0)
        pred = np.array(pred_logits.argmax(axis=0),)*255
        mask[x : x + pixel_shape, y : y + pixel_shape] = pred

mask = Image.fromarray(mask[:origin_h,:origin_w].astype(np.uint8))
mask.save(r'D:\31890\Desktop\codefile\Utils\Packages\onnx__\test2.png')

time_end = time.time()
print("time cost:", time_end-time_start , "s")

100%|██████████| 27/27 [17:01<00:00, 37.82s/it]


time cost: 1026.4172325134277 s
